# 多元線性回歸 Multiple Linear Regression

## 要做多元線性回歸有 5 個限定條件：
1. Linearity 線性
2. Homoscedasticity 同方差性
3. Multivariate Normality 多元正態分佈
4. Independence of errors 誤差獨立
5. Lack of multicollinearity 無多重共線性

In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### 投資人想知道哪間新創公司是會賺錢的

In [62]:
# Importing the dataset
dataset = pd.read_csv('50_Startups.csv')
X = dataset.iloc[:, :-1].values #研發開支、行政開支、行銷開支、公司地點
y = dataset.iloc[:, 4].values #獲利(收益)

In [4]:
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


### 分類數據與虛擬變量
* 這幾個自變量中，State不具有連續，也無法進行比較(排序)，屬於"分類數據"
* 我們可將分類數據轉換成"虛擬變量"，一般習慣用0和1來表示

In [63]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 3] = labelencoder_X.fit_transform(X[:, 3]) #把State這個自變量轉換成分類數據

In [64]:
X[:10,3] #看一下X現在長怎樣(前十筆)

array([2, 0, 1, 2, 1, 2, 0, 1, 2, 0], dtype=object)

In [65]:
# Encoding the Independent Variable
onehotencoder = OneHotEncoder(categorical_features = [3]) #指定State這個自變量
X = onehotencoder.fit_transform(X).toarray() #依照此自變量的分類數，產生相對個數的虛擬變量

In [68]:
X[:10,0:3] #看一下X現在長怎樣(前十筆)

array([[  0.00000000e+00,   1.00000000e+00,   1.65349200e+05],
       [  0.00000000e+00,   0.00000000e+00,   1.62597700e+05],
       [  1.00000000e+00,   0.00000000e+00,   1.53441510e+05],
       [  0.00000000e+00,   1.00000000e+00,   1.44372410e+05],
       [  1.00000000e+00,   0.00000000e+00,   1.42107340e+05],
       [  0.00000000e+00,   1.00000000e+00,   1.31876900e+05],
       [  0.00000000e+00,   0.00000000e+00,   1.34615460e+05],
       [  1.00000000e+00,   0.00000000e+00,   1.30298130e+05],
       [  0.00000000e+00,   1.00000000e+00,   1.20542520e+05],
       [  0.00000000e+00,   0.00000000e+00,   1.23334880e+05]])

### 虛擬變量陷阱
* 為避免自變量當中有多重共線性(任一自變量不能是其他自變量的線性組合)，例如：D2=1-D1就是顯然明確的關係
* 參數過多會有過度擬合的問題，且參數夠用就好了(n-1即可表示所有情況)
#### 小結：虛擬變量有n個，我們就只需要用到n-1個即可

![dummy variable trap](dummy_variable_trap.png)

In [67]:
# Avoiding the Dummy Variable Trap
X = X[:, 1:] #三個虛擬變量，要移除哪一個都可以

In [69]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### 接下來跟簡單線性回歸一樣，只是多了幾個自變量

In [70]:
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
regr = LinearRegression()
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [71]:
# Predicting the Test set results
y_pred = regr.predict(X_test)

### 五種方法讓我們找到一個好模型
* All-in
* Backward Elimination 反向淘汰 (推)
* Forward Selection 順向選擇 
* Bidirectional Elimination 雙向淘汰
* Score Comparison 信息量比較

### 反向淘汰
* 先定義"顯著性門檻"，一開始先用All-in的方式把所有自變量進行擬合
* 透過統計的得到每個自變量在模型下的P值，P值越高代表對此模型的顯著性越低
* 每次淘汰掉P值最高且大於顯著性門檻的自變量，最後剩下的組合即是反向淘汰找出的最佳模型

![backward_elimination](backward_elimination.png)

### 多元線性回歸函數中的常數像其實也是一個參數，可以當做 y = b0*1 + b1*x1 + b2*x2 + ... 
* 所以我們要在X_train中增加一行(第一欄)都是1的矩陣
* np.append函數把values加到arr後面，axis即增加行

In [72]:
# Building the optimal model using Backward Elimination
import statsmodels.formula.api as sm
X_train = np.append(arr = np.ones((40, 1)).astype(int), values = X_train, axis = 1)

### 逐步地將不同自變量組合進行擬合，並查看不同組合產生的模型表現

In [75]:
X_opt = X_train [:, [0, 1, 2, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary() #可以看到x2有最高的P值，且高於我們定的門檻值(SL=0.05)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     129.7
Date:                Fri, 27 Apr 2018   Prob (F-statistic):           3.91e-21
Time:                        00:02:57   Log-Likelihood:                -421.10
No. Observations:                  40   AIC:                             854.2
Df Residuals:                      34   BIC:                             864.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.255e+04   8358.538      5.091      0.000    2.56e+04    5.95e+04
x1          -959.2842   4038.108     -0.238      0.814   -9165.706    7247.138
x2           699.3691   3661.563      0.191      0.850   -6741.822    8140.560
x3             0.7735      0.055     14.025      0.000       0.661       0.886
x4             0.0329      0.066      0.495      0.624      -0.102       0.168
x5             0.0366      0.019      1.884      0.068      -0.003       0.076
==============================================================================
Omnibus:                       15.823   Durbin-Watson:                   2.468
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.231
Skew:                          -1.094   Prob(JB):                     9.03e-06
Kurtosis:                       6.025   Cond. No.                     1.49e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.49e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [76]:
X_opt = X_train [:, [0, 1, 3, 4, 5]] #把x2踢除
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     166.7
Date:                Fri, 27 Apr 2018   Prob (F-statistic):           2.87e-22
Time:                        00:04:18   Log-Likelihood:                -421.12
No. Observations:                  40   AIC:                             852.2
Df Residuals:                      35   BIC:                             860.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.292e+04   8020.397      5.352      0.000    2.66e+04    5.92e+04
x1         -1272.1608   3639.780     -0.350      0.729   -8661.308    6116.986
x2             0.7754      0.053     14.498      0.000       0.667       0.884
x3             0.0319      0.065      0.488      0.629      -0.101       0.165
x4             0.0363      0.019      1.902      0.065      -0.002       0.075
==============================================================================
Omnibus:                       16.074   Durbin-Watson:                   2.467
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               24.553
Skew:                          -1.086   Prob(JB):                     4.66e-06
Kurtosis:                       6.164   Cond. No.                     1.43e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.43e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [77]:
X_opt = X_train [:, [0, 3, 4, 5]] #把x1
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     227.8
Date:                Fri, 27 Apr 2018   Prob (F-statistic):           1.85e-23
Time:                        00:04:36   Log-Likelihood:                -421.19
No. Observations:                  40   AIC:                             850.4
Df Residuals:                      36   BIC:                             857.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.299e+04   7919.773      5.428      0.000    2.69e+04    5.91e+04
x1             0.7788      0.052     15.003      0.000       0.674       0.884
x2             0.0294      0.064      0.458      0.650      -0.101       0.160
x3             0.0347      0.018      1.896      0.066      -0.002       0.072
==============================================================================
Omnibus:                       15.557   Durbin-Watson:                   2.481
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               22.539
Skew:                          -1.081   Prob(JB):                     1.28e-05
Kurtosis:                       5.974   Cond. No.                     1.43e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.43e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [78]:
X_opt = X_train [:, [0, 3, 5]] #把x2踢除
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.947
Method:                 Least Squares   F-statistic:                     349.0
Date:                Fri, 27 Apr 2018   Prob (F-statistic):           9.65e-25
Time:                        00:07:50   Log-Likelihood:                -421.30
No. Observations:                  40   AIC:                             848.6
Df Residuals:                      37   BIC:                             853.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.635e+04   2971.236     15.598      0.000    4.03e+04    5.24e+04
x1             0.7886      0.047     16.846      0.000       0.694       0.883
x2             0.0326      0.018      1.860      0.071      -0.003       0.068
==============================================================================
Omnibus:                       14.666   Durbin-Watson:                   2.518
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               20.582
Skew:                          -1.030   Prob(JB):                     3.39e-05
Kurtosis:                       5.847   Cond. No.                     4.97e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.97e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [79]:
X_opt = X_train [:, [0, 3]] #把x2踢除
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.945
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     652.4
Date:                Fri, 27 Apr 2018   Prob (F-statistic):           1.56e-25
Time:                        00:08:03   Log-Likelihood:                -423.09
No. Observations:                  40   AIC:                             850.2
Df Residuals:                      38   BIC:                             853.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.842e+04   2842.717     17.032      0.000    4.27e+04    5.42e+04
x1             0.8516      0.033     25.542      0.000       0.784       0.919
==============================================================================
Omnibus:                       13.132   Durbin-Watson:                   2.325
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               16.254
Skew:                          -0.991   Prob(JB):                     0.000295
Kurtosis:                       5.413   Cond. No.                     1.57e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.57e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## 結論：
* 結果研發開支其實是影響這些公司最後收益的關鍵(行銷開支也算啦~
* 多項式線性回歸中，參數越多不見得會比較好，可能關鍵的就只有某幾個
* 可透過各種方法來找出最關鍵的幾個參數，也代表找到各方法下最佳的模型
* 但沒有最好只有更好，後續會講解如何更客觀地評估一個模型的表現

## 回歸模型性能評價及選擇
### R平方：
* 是0~1之間的數值，值越大代表準確率越高，也就是模型越好
* 自變量數量增加時，R平方必定會變大或是維持不定
* 但這不是個好方法，所以就提出"Adjusted R平方"

### Adjusted R平方：
* 讓自變量的數量有懲戒的效果(變小)，所以自變量數量增加時，不一定讓"Adjusted R平方"變高或變低
* 但是這種方法，可以更客觀地評估(不是主觀定義一個閥值)

![adjusted_r2](adjusted_r2.png)